In [1]:
for i in range(10):
    print("{0:06d}".format(i+1))

000001
000002
000003
000004
000005
000006
000007
000008
000009
000010


In [ ]:
import numpy as np


 = np.load(os.path.join(root_dir, 'audio', 'folder_count.npy')).tolist()